In [29]:
# System libraries:
%load_ext autoreload
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARN)
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pickle
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# Train word2vec models starting at each year so around 15-20 models
data = pd.read_csv('data/processed/yle_fi_lemmas_new.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
data.iloc[:5]

,id,headline,content,timePublished,lemmas_content,year
0,0,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,2008-08-11T11:03:41+0300,"[kaukasia, konflikti, asi, georg, venäjä, tuke...",2008
1,1,Otteita Venäjän-tuntijoiden vastauksista,Otteita Venäjän-tuntijoiden vastauksista\n\nTä...,2007-01-14T12:46:58+0200,"[ote, venäjätuntija, vastaus, tiivistelmä, koo...",2007
2,2,Etelä-Ossetian ruutitynnyri räjähti viimein,Etelä-Ossetian ruutitynnyri räjähti viimein\n\...,2008-08-12T21:33:32+0300,"[etelä-ossetia, ruutitynnyri, räjähtää, viimei...",2008
3,3,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,2014-03-18T09:06:59+0200,"[yle, uutinen, seurata, ukraina, kriisi, hetki...",2014
4,4,Saksan ulkopolitiikan täyskäännös,Saksan ulkopolitiikan täyskäännös\n\nTuskin mi...,2014-10-02T12:30:30+0300,"[saksa, ulkopolitiikka, täyskäännös, tuskin, m...",2014


In [31]:
def create_models_for_multiple_years(corpus):
    years = sorted(corpus.year.unique())
    models = {}
    for i, year in enumerate(years[:-4]):
        print("Building model for " +str(year))
        sentences = corpus[corpus.year.isin(years[i:i+5])].lemmas_content
        models[year] = gensim.models.Word2Vec(sentences, min_count=1)
        
    sentences = corpus.lemmas_content
    models['all'] = gensim.models.Word2Vec(sentences, min_count=1)
    
    return models
    

models = create_models_for_multiple_years(data)

Building model for 1997
Building model for 1998
Building model for 1999
Building model for 2000
Building model for 2001
Building model for 2002
Building model for 2003
Building model for 2004
Building model for 2005
Building model for 2006
Building model for 2007
Building model for 2008
Building model for 2009
Building model for 2010
Building model for 2011
Building model for 2012
Building model for 2013


In [7]:
#pickle_path = 'pickles/yle_word_models.pickle'
#pickle.dump(models, open(pickle_path, "wb"))

In [ ]:
#serialize models with pickle

In [32]:
def get_most_similar(models, words, n):
    results = {}
    for year in models:
        results[year] = models[year].wv.most_similar(positive=words, topn=n)
        
    return results

def compare_words(models, word1, word2):
    results = {}
    for year in models:
        results[year] = models[year].wv.similarity(word1, word2)
        
    return results

def word_vector_math(models, positive_terms, negative_terms, n):
    results = {}
    for year in models:
        results[year] = models[year].wv.most_similar(positive=positive_terms, 
                                                      negative=negative_terms, topn=n)
    
    return results

In [34]:
# Get words most similar to a topic
words = ['venäjä']
number_results = 10

get_most_similar(models, words, number_results)

{1997: [('maahanmuuttoviranomainen', 0.967134416103363),
  ('japan', 0.9495872855186462),
  ('aidsrahasto', 0.9441395998001099),
  ('viro', 0.94366455078125),
  ('ympäristöyhteistyö', 0.9435255527496338),
  ('kiina', 0.9411607980728149),
  ('saapua', 0.9393817186355591),
  ('tennisliitto', 0.9392222166061401),
  ('ohjuskoe', 0.9365344047546387),
  ('yhteinenkomissio', 0.9364097118377686)],
 1998: [('viro', 0.8533923625946045),
  ('kiina', 0.8412581086158752),
  ('pää-markkina-alue', 0.8405821919441223),
  ('vahingottaa', 0.8222353458404541),
  ('luotonsaaja', 0.8145128488540649),
  ('keskinen', 0.8059436082839966),
  ('paikallisjohtaja', 0.8056646585464478),
  ('vieraisilla', 0.8054907321929932),
  ('hongkong', 0.8051058053970337),
  ('liikennesopimus', 0.8034248948097229)],
 1999: [('viro', 0.744361937046051),
  ('liettua', 0.7340520024299622),
  ('neuvostoliitto', 0.7265894412994385),
  ('japan', 0.7184039950370789),
  ('koillis-aasia', 0.71040278673172),
  ('ihmisoikeustuomioistuin'

In [26]:
# Get the distance between words over years, bigger is closer:
word1 = 'naapuri'
word2 = 'hyvä'

compare_words(models, word1, word2)

{1997: 0.4587858354241383,
 1998: 0.2756432962098212,
 1999: 0.24622687787875888,
 2000: 0.1941150963025495,
 2001: 0.10008476600604832,
 2002: 0.0796201266477919,
 2003: 0.14264070700543302,
 2004: 0.049155215792477745,
 2005: 0.10637148449483241,
 2006: 0.11323065067370557,
 2007: 0.18493518621236527,
 2008: 0.18977124305855406,
 2009: 0.17671510184941228,
 2010: 0.20609463508488657,
 2011: 0.2553885168157117,
 2012: 0.2475238195808103,
 2013: 0.29418588017212083,
 'all': 0.20846976412825405}

In [28]:
# Calculate the vector math - "what is X to Russia as Sauna is to Finland":
positive_terms = ['venäjä', 'naapuri']
negative_terms = ['suomi']

number_results = 3

word_vector_math(models, positive_terms, negative_terms, number_results)


{1997: [('tshetshenia', 0.9314170479774475),
  ('parlamentti', 0.8991386890411377),
  ('dagestan', 0.8985882997512817)],
 1998: [('rice', 0.8421714305877686),
  ('serbia', 0.8411684036254883),
  ('tshetshenia', 0.8402163982391357)],
 1999: [('georgia', 0.8186936974525452),
  ('taiwan', 0.799132227897644),
  ('toistuvasti', 0.79735267162323)],
 2000: [('kaksinainen', 0.7719497680664062),
  ('intia', 0.7535704970359802),
  ('gougne', 0.7526333928108215)],
 2001: [('pakistan', 0.7591776847839355),
  ('intia', 0.7334529757499695),
  ('taiwan', 0.7296583652496338)],
 2002: [('pakistan', 0.721416711807251),
  ('taiwan', 0.7146213054656982),
  ('xinjiang', 0.6781677603721619)],
 2003: [('länsi', 0.6718704700469971),
  ('myanmar', 0.6506058573722839),
  ('viha', 0.6447268724441528)],
 2004: [('länsi', 0.6382226943969727),
  ('kirgisia', 0.6373786330223083),
  ('viha', 0.6124135851860046)],
 2005: [('länsi', 0.6214760541915894),
  ('tahra', 0.5833086967468262),
  ('pakistan', 0.5665426254272461